# Exploratory notebook for IMACLIM within MATMat-Trade

## Creating the MatMat-Trade model base on IMACLIM regions and sectors but on EXIOBASE 3 data

In [1]:
# for some reason saved model has issues with the names of regions so we rebuild the model each times

import shutil
shutil.rmtree('data/models/2015__pxp__IMACLIM_format__ghg')

In [2]:
from main import Model
import pandas as pd

model = Model(aggregation_name="IMACLIM_format")

Loading data... (may take a few minutes)
Data loaded successfully !


## Creating the scenarios based on IMACLIM scenarios and the projected changes 

In [3]:
#ignore performace warnings
import warnings

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [4]:
import tqdm

from src.scenarios import DICT_SCENARIOS
scenario_list=["INDC","INDC2020_1000","INDC2030_1000","INDC2020_1600","INDC2020_400","INDC2030_1600","baseline"]

for scenario in tqdm.tqdm(scenario_list[:1]):
    # create 2050 version of each scenarios applying emissivity changes and then technological changes
    model.new_counterfactual("IMACLIM_all_2050_"+scenario, DICT_SCENARIOS["emissivity_IMACLIM"], year=2050,scenario=scenario)
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["technical_change_IMACLIM"], year=2050,scenario=scenario)
    # create the 2050 version based on production changes observed in IMACLIM (since we work on groos output, we need a reference to apply relative change : x_ref), with name new_name
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["production_change_IMACLIM"], year=2050,scenario="INDC",new_name="IMACLIM_all_prod_change_2050_"+scenario,x_ref=model.iot.x.sort_index())
    
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["consumption_change_imaclim"], year=2050,scenario="INDC",new_name="IMACLIM_all_cons_change_2050_"+scenario)

  0%|          | 0/1 [00:02<?, ?it/s]


KeyboardInterrupt: 

## Getting the emissions per types of scenario 

In [6]:
from src.utils import footprint_extractor
scenario_list=["INDC","INDC2020_1000","INDC2030_1000","INDC2020_1600","INDC2020_400","INDC2030_1600","baseline"]

start_year=pd.concat([ pd.concat([pd.DataFrame(footprint_extractor(model,region=region),index=[2015]).T  for region in model.regions],
                      keys=model.regions,
                      names=("regions","types")) for scenario in scenario_list],
          keys=scenario_list,
          names=("scenario","regions","types"))

Emissions=pd.concat([start_year for i in range(4)],axis=1,keys=["2015","2050","2050-Prod","2050-Cons"])
Emissions.columns=["2015","2050","2050-Prod","2050-Cons"]



for scenario in tqdm.tqdm(scenario_list):
    # create 2050 version of each scenarios applying emissivity changes and then technological changes
    model.new_counterfactual("IMACLIM_all_2050_"+scenario, DICT_SCENARIOS["emissivity_IMACLIM"], year=2050,scenario=scenario)
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["technical_change_IMACLIM"], year=2050,scenario=scenario)
    # create the 2050 version based on production changes observed in IMACLIM (since we work on groos output, we need a reference to apply relative change : x_ref), with name new_name
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["production_change_IMACLIM"], year=2050,scenario="INDC",new_name="IMACLIM_all_prod_change_2050_"+scenario,x_ref=model.iot.x.sort_index())
    
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["consumption_change_imaclim"], year=2050,scenario="INDC",new_name="IMACLIM_all_cons_change_2050_"+scenario)
    
    Emissions.loc[(scenario),"2050"]=pd.concat([pd.DataFrame(footprint_extractor(model.counterfactuals["IMACLIM_all_2050_"+scenario],region=region),index=[2050]).T  for region in model.regions],
                        keys=model.regions,
                        names=("regions","types")).values
    
    Emissions.loc[(scenario),"2050-Prod"]=pd.concat([pd.DataFrame(footprint_extractor(model.counterfactuals["IMACLIM_all_prod_change_2050_"+scenario],region=region),index=[2050]).T  for region in model.regions],
                        keys=model.regions,
                        names=("regions","types")).values
    
    Emissions.loc[(scenario),"2050-Cons"]=pd.concat([pd.DataFrame(footprint_extractor(model.counterfactuals["IMACLIM_all_cons_change_2050_"+scenario],region=region),index=[2050]).T  for region in model.regions],
                        keys=model.regions,
                        names=("regions","types")).values
    
    del(model.counterfactuals["IMACLIM_all_2050_"+scenario])
    del(model.counterfactuals["IMACLIM_all_cons_change_2050_"+scenario])
    del(model.counterfactuals["IMACLIM_all_prod_change_2050_"+scenario])
    
    

  0%|          | 0/7 [00:00<?, ?it/s]

Problems on sums of columns  Series([], dtype: float64)


 14%|█▍        | 1/7 [00:53<05:20, 53.34s/it]

Problems on sums of columns  region  sector               
AFR     Terrestrial transport     186.058374
MDE     Terrestrial transport    2412.591216
RAL     Terrestrial transport     128.913672
dtype: float64


 29%|██▊       | 2/7 [01:48<04:30, 54.14s/it]

...

 71%|███████▏  | 5/7 [04:37<01:53, 56.63s/it]

Problems on sums of columns  region  sector               
AFR     Terrestrial transport      94.507170
MDE     Terrestrial transport    1088.774681
RAL     Terrestrial transport      67.292792
dtype: float64


 86%|████████▌ | 6/7 [05:34<00:56, 56.69s/it]

Problems on sums of columns  Series([], dtype: float64)


100%|██████████| 7/7 [06:36<00:00, 56.71s/it]


In [18]:
Emissions.loc["INDC2030_1600"].sum(axis=0,level=1)

,2015,2050,2050-Prod,2050-Cons
types,,,,
Exportations,-1.043718e+13,-5.862308e+12,-1.644826e+13,-1.817036e+13
Production,4.145956e+13,1.945288e+13,5.730547e+13,6.368677e+13
Importations,1.043718e+13,5.862308e+12,1.644826e+13,1.817036e+13
Consommation,5.524603e+12,5.524603e+12,2.005752e+13,1.701417e+13


In [33]:
Emissions.loc[(slice(None),slice(None),"Production")]

2015          2050     2050-Prod  \
scenario regions types                                                  
INDC     AFR     Production  2.412825e+12  2.035437e+12  1.334894e+13   
         BRA     Production  1.085191e+12  8.746764e+11  2.511632e+12   
         CAN     Production  6.529151e+11  3.992860e+11  1.047203e+12   
         CHN     Production  1.151045e+13  4.664359e+12  1.264150e+13   
         CIS     Production  1.846605e+12  1.216074e+12  3.833561e+12   
...                                   ...           ...           ...   
baseline JAN     Production  1.867037e+12  1.291313e+12  2.115655e+12   
         MDE     Production  2.579607e+12  1.820303e+12  5.887259e+12   
         RAL     Production  1.958445e+12  1.637901e+12  3.763869e+12   
         RAS     Production  4.682540e+12  3.317406e+12  1.198517e+13   
         USA     Production  4.829634e+12  3.224305e+12  6.276607e+12   

                                2050-Cons  
scenario regions types                     
INDC     AFR     Production  8.313479e+12  
         BRA     Production  2.560068e+12  
         CAN     Production  1.193691e+12  
         CHN     Production  1.580910e+13  
         CIS     Production  3.711176e+12  
...                                   ...  
baseline JAN     Production  3.236555e+12  
         MDE     Production  6.545419e+12  
         RAL     Production  4.463721e+12  
         RAS     Production  1.167476e+13  
         USA     Production  8.146692e+12  

[84 rows x 4 columns]

In [19]:
import src.advance
final_data_ratio,final_technical_coef,Link_country,Link,Production_volumes,total_consumption,emissions=src.advance.extract_data("IMACLIM_format")

In [34]:
emissions[2050].groupby(["Scenario","Region"]).sum()*1e9

Scenario  Region
INDC      AFR       4.826130e+12
          BRA       5.722192e+11
          CAN       4.409896e+11
          CHN       3.914820e+12
          CIS       4.070680e+12
                        ...     
baseline  MDE       3.090390e+12
          RAL       1.851780e+12
          RAS       4.850239e+12
          USA       5.596570e+12
          World     4.238270e+13
Name: 2050, Length: 91, dtype: float64

In [25]:
model.iot.stressor_extension.S.dot(model.iot.Y)

region                                           AFR  \
category Final consumption expenditure by households   
CO2                                     2.785334e+11   
CH4                                     3.480231e+11   
N2O                                     1.071688e+11   
SF6                                     8.037921e+08   
HFC                                     2.555006e+09   
PFC                                     8.225578e+08   

region                                                                                         \
category Final consumption expenditure by non-profit organisations serving households (NPISH)   
CO2                                            4.787910e+09                                     
CH4                                            6.822239e+09                                     
N2O                                            1.389551e+09                                     
SF6                                            6.785561e+06                                     
HFC                                            2.639568e+07                                     
PFC                                            1.597531e+06                                     

region                                                \
category Final consumption expenditure by government   
CO2                                     1.538574e+10   
CH4                                     2.228198e+10   
N2O                                     4.292760e+09   
SF6                                     2.793052e+07   
HFC                                     9.174403e+07   
PFC                                     1.506917e+07   

region                                                         \
category Gross fixed capital formation Changes in inventories   
CO2                       9.877345e+10           2.866982e+09   
CH4                       7.166073e+10           3.750995e+09   
N2O                       1.130743e+10           3.752828e+08   
SF6                       3.010710e+08           7.187029e+06   
HFC                       1.012377e+09           1.426768e+07   
PFC                       3.104964e+08           7.797248e+06   

region                                              \
category Changes in valuables Exports: Total (fob)   
CO2              3.800544e+09                  0.0   
CH4              3.977167e+07                  0.0   
N2O              1.688590e+07                  0.0   
SF6              7.980692e+04                  0.0   
HFC              1.516459e+05                  0.0   
PFC              4.005660e+03                  0.0   

region                                           BRA  \
category Final consumption expenditure by households   
CO2                                     1.143294e+11   
CH4                                     1.274554e+11   
N2O                                     3.447924e+10   
SF6                                     1.092944e+08   
HFC                                     2.041571e+09   
PFC                                     8.751837e+08   

region                                                                                         \
category Final consumption expenditure by non-profit organisations serving households (NPISH)   
CO2                                            1.221015e+09                                     
CH4                                            1.512048e+07                                     
N2O                                            1.000969e+07                                     
SF6                                            8.362872e+04                                     
HFC                                            2.104700e+06                                     
PFC                                            3.774773e+04                                     

region                                                ...  \
category Final consumption expenditure by government  ...   
CO2                    

AttributeError: 'IOSystem' object has no attribute 'stressor_extension'

In [51]:
scenario="INDC"
model.new_counterfactual("IMACLIM_all_2050_"+scenario, DICT_SCENARIOS["emissivity_IMACLIM"], year=2050,scenario=scenario)
model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["technical_change_IMACLIM"], year=2050,scenario=scenario)

Problems on sums of columns  Series([], dtype: float64)


In [58]:
model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["production_change_IMACLIM"], year=2050,scenario="INDC",new_name="IMACLIM_all_prod_change_2050_"+scenario,x_ref=model.iot.x.sort_index())
    
model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["consumption_change_imaclim"], year=2050,scenario="INDC",new_name="IMACLIM_all_cons_change_2050_"+scenario)
    

In [59]:
from pymrio.tools.iomath import calc_F_Y
F_Y=calc_F_Y(model.counterfactuals["IMACLIM_all_cons_change_2050_INDC"].iot.stressor_extension.S_Y,model.counterfactuals["IMACLIM_all_cons_change_2050_INDC"].iot.Y.sum())

In [60]:
F_Y

region                                           AFR  \
category Final consumption expenditure by households   
CO2                                     1.369166e+12   
CH4                                     4.337297e+11   
N2O                                     6.377476e+10   
SF6                                     0.000000e+00   
HFC                                     0.000000e+00   
PFC                                     0.000000e+00   

region                                                                                         \
category Final consumption expenditure by non-profit organisations serving households (NPISH)   
CO2                                            1.063049e+08                                     
CH4                                            1.259803e+06                                     
N2O                                            9.781622e+04                                     
SF6                                            0.000000e+00                                     
HFC                                            0.000000e+00                                     
PFC                                            0.000000e+00                                     

region                                                \
category Final consumption expenditure by government   
CO2                                     2.834543e+09   
CH4                                     1.658924e+09   
N2O                                     2.236637e+08   
SF6                                     0.000000e+00   
HFC                                     0.000000e+00   
PFC                                     0.000000e+00   

region                                                         \
category Gross fixed capital formation Changes in inventories   
CO2                                0.0                    0.0   
CH4                                0.0                    0.0   
N2O                                0.0                    0.0   
SF6                                0.0                    0.0   
HFC                                0.0                    0.0   
PFC                                0.0                    0.0   

region                                              \
category Changes in valuables Exports: Total (fob)   
CO2                       0.0                  0.0   
CH4                       0.0                  0.0   
N2O                       0.0                  0.0   
SF6                       0.0                  0.0   
HFC                       0.0                  0.0   
PFC                       0.0                  0.0   

region                                           BRA  \
category Final consumption expenditure by households   
CO2                                     2.441102e+11   
CH4                                     1.124609e+10   
N2O                                     4.881682e+09   
SF6                                     0.000000e+00   
HFC                                     0.000000e+00   
PFC                                     0.000000e+00   

region                                                                                         \
category Final consumption expenditure by non-profit organisations serving households (NPISH)   
CO2                                            1.104983e+08                                     
CH4                                            1.362729e+07                                     
N2O                                            1.502983e+05                                     
SF6                                            0.000000e+00                                     
HFC                                            0.000000e+00                                     
PFC                                            0.000000e+00                                     

region                                                ...  \
category Final consumption expenditure by government  ...   
CO2                    

In [57]:
model.counterfactuals["IMACLIM_all_2050_INDC"].iot.stressor_extension.F_Y

region                                           AFR  \
category Final consumption expenditure by households   
CO2                                     3.116139e+11   
CH4                                     9.871424e+10   
N2O                                     1.451475e+10   
SF6                                     0.000000e+00   
HFC                                     0.000000e+00   
PFC                                     0.000000e+00   

region                                                                                         \
category Final consumption expenditure by non-profit organisations serving households (NPISH)   
CO2                                            2.419435e+07                                     
CH4                                            2.867235e+05                                     
N2O                                            2.226237e+04                                     
SF6                                            0.000000e+00                                     
HFC                                            0.000000e+00                                     
PFC                                            0.000000e+00                                     

region                                                \
category Final consumption expenditure by government   
CO2                                     6.451246e+08   
CH4                                     3.775609e+08   
N2O                                     5.090450e+07   
SF6                                     0.000000e+00   
HFC                                     0.000000e+00   
PFC                                     0.000000e+00   

region                                                         \
category Gross fixed capital formation Changes in inventories   
CO2                                0.0                    0.0   
CH4                                0.0                    0.0   
N2O                                0.0                    0.0   
SF6                                0.0                    0.0   
HFC                                0.0                    0.0   
PFC                                0.0                    0.0   

region                                              \
category Changes in valuables Exports: Total (fob)   
CO2                       0.0                  0.0   
CH4                       0.0                  0.0   
N2O                       0.0                  0.0   
SF6                       0.0                  0.0   
HFC                       0.0                  0.0   
PFC                       0.0                  0.0   

region                                           BRA  \
category Final consumption expenditure by households   
CO2                                     8.477625e+10   
CH4                                     3.905620e+09   
N2O                                     1.695344e+09   
SF6                                     0.000000e+00   
HFC                                     0.000000e+00   
PFC                                     0.000000e+00   

region                                                                                         \
category Final consumption expenditure by non-profit organisations serving households (NPISH)   
CO2                                            3.837461e+07                                     
CH4                                            4.732580e+06                                     
N2O                                            5.219661e+04                                     
SF6                                            0.000000e+00                                     
HFC                                            0.000000e+00                                     
PFC                                            0.000000e+00                                     

region                                                ...  \
category Final consumption expenditure by government  ...   
CO2                    

In [8]:
model.iot